<a href="https://colab.research.google.com/github/monsieurlx/Feedback-Network/blob/main/notebookFeedbackNetPytorchLightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install -qqq wandb pytorch-lightning

In [ ]:
# PyTorch Lightning
import pytorch_lightning as pl
pl.seed_everything(hash("setting random seeds") % 2**32 - 1)

# Weights & Biases
import wandb


from pytorch_lightning.loggers import WandbLogger

wandb.login()

Global seed set to 2217166214


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Definition of ConvLSTMCell

In [ ]:
import torch
import pytorch_lightning as pl
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import math


class ConvLSTMCell(pl.LightningModule):
    
    def __init__(self, input_size, output_size, x_kernel_size, h_kernel_size, stride=1, num_cell=3):
        super(ConvLSTMCell, self).__init__()
        pad_x = int(math.floor(x_kernel_size/2))
        pad_h = int(math.floor(h_kernel_size/2))
        self.output_size = output_size
        self.stride = stride
        
        # input gate
        conv_i_x = nn.Conv2d(input_size, output_size, x_kernel_size, stride=stride, padding=pad_x)
        batchnorm_i_x = nn.BatchNorm2d(output_size)
        self.i_x = nn.Sequential()
        self.i_x.add_module('conv_1_x', conv_i_x)
        self.i_x.add_module('bn_1_x', batchnorm_i_x)
        for i in range(num_cell-1):
            relu_name = 'relu' + str(i+1)
            conv_name = 'conv_{}_x'.format(str(i + 2))
            bn_name = 'bn_{}_x'.format(str(i + 2))
            self.i_x.add_module(relu_name, nn.ReLU())
            self.i_x.add_module(conv_name, nn.Conv2d(output_size, output_size, x_kernel_size, stride=1, padding=pad_x))
            self.i_x.add_module(bn_name, nn.BatchNorm2d(output_size))

        conv_i_h = nn.Conv2d(output_size, output_size, h_kernel_size, stride=1, padding=pad_h)
        batchnorm_i_h = nn.BatchNorm2d(output_size)
        self.i_h = nn.Sequential()
        self.i_h.add_module('conv_1_h', conv_i_h)
        self.i_h.add_module('bn_1_h', batchnorm_i_h)
        for i in range(num_cell-1):
            relu_name = 'relu' + str(i+1)
            conv_name = 'conv_{}_h'.format(str(i + 2))
            bn_name = 'bn_{}_h'.format(str(i + 2))
            self.i_h.add_module(relu_name, nn.ReLU())
            self.i_h.add_module(conv_name, nn.Conv2d(output_size, output_size, h_kernel_size, stride=1, padding=pad_h))
            self.i_h.add_module(bn_name, nn.BatchNorm2d(output_size))
        
       

        conv_f_x = nn.Conv2d(input_size, output_size, x_kernel_size, stride=stride, padding=pad_x)
        batchnorm_f_x = nn.BatchNorm2d(output_size)
        self.f_x = nn.Sequential()
        self.f_x.add_module('conv_1_x', conv_f_x)
        self.f_x.add_module('bn_1_x', batchnorm_f_x)
        for i in range(num_cell-1):
            relu_name = 'relu' + str(i+1)
            conv_name = 'conv_{}_x'.format(str(i + 2))
            bn_name = 'bn_{}_x'.format(str(i + 2))
            self.f_x.add_module(relu_name, nn.ReLU())
            self.f_x.add_module(conv_name, nn.Conv2d(output_size, output_size, x_kernel_size, stride=1, padding=pad_x))
            self.f_x.add_module(bn_name, nn.BatchNorm2d(output_size))

        conv_f_h = nn.Conv2d(output_size, output_size, h_kernel_size, stride=1, padding=pad_h)
        batchnorm_f_h = nn.BatchNorm2d(output_size)
        self.f_h = nn.Sequential()
        self.f_h.add_module('conv_1_h', conv_f_h)
        self.f_h.add_module('bn_1_h', batchnorm_f_h)
        for i in range(num_cell-1):
            relu_name = 'relu' + str(i+1)
            conv_name = 'conv_{}_h'.format(str(i + 2))
            bn_name = 'bn_{}_h'.format(str(i + 2))
            self.f_h.add_module(relu_name, nn.ReLU())
            self.f_h.add_module(conv_name, nn.Conv2d(output_size, output_size, h_kernel_size, stride=1, padding=pad_h))
            self.f_h.add_module(bn_name, nn.BatchNorm2d(output_size))
        
      
        conv_c_x = nn.Conv2d(input_size, output_size, x_kernel_size, stride=stride, padding=pad_x)
        batchnorm_c_x = nn.BatchNorm2d(output_size)
        self.c_x = nn.Sequential()
        self.c_x.add_module('conv_1_x', conv_c_x)
        self.c_x.add_module('bn_1_x', batchnorm_c_x)
        for i in range(num_cell-1):
            relu_name = 'relu' + str(i+1)
            conv_name = 'conv_{}_x'.format(str(i + 2))
            bn_name = 'bn_{}_x'.format(str(i + 2))
            self.c_x.add_module(relu_name, nn.ReLU())
            self.c_x.add_module(conv_name, nn.Conv2d(output_size, output_size, x_kernel_size, stride=1, padding=pad_x))
            self.c_x.add_module(bn_name, nn.BatchNorm2d(output_size))

        conv_c_h = nn.Conv2d(output_size, output_size, h_kernel_size, stride=1, padding=pad_h)
        batchnorm_c_h = nn.BatchNorm2d(output_size)
        self.c_h = nn.Sequential()
        self.c_h.add_module('conv_1_h', conv_c_h)
        self.c_h.add_module('bn_1_h', batchnorm_c_h)
        for i in range(num_cell-1):
            relu_name = 'relu' + str(i+1)
            conv_name = 'conv_{}_h'.format(str(i + 2))
            bn_name = 'bn_{}_h'.format(str(i + 2))
            self.c_h.add_module(relu_name, nn.ReLU())
            self.c_h.add_module(conv_name, nn.Conv2d(output_size, output_size, h_kernel_size, stride=1, padding=pad_h))
            self.c_h.add_module(bn_name, nn.BatchNorm2d(output_size))

       
        conv_o_x = nn.Conv2d(input_size, output_size, x_kernel_size, stride=stride, padding=pad_x)
        batchnorm_o_x = nn.BatchNorm2d(output_size)
        self.o_x = nn.Sequential()
        self.o_x.add_module('conv_1_x', conv_o_x)
        self.o_x.add_module('bn_1_x', batchnorm_o_x)
        for i in range(num_cell-1):
            relu_name = 'relu' + str(i+1)
            conv_name = 'conv_{}_x'.format(str(i + 2))
            bn_name = 'bn_{}_x'.format(str(i + 2))
            self.o_x.add_module(relu_name, nn.ReLU())
            self.o_x.add_module(conv_name, nn.Conv2d(output_size, output_size, x_kernel_size, stride=1, padding=pad_x))
            self.o_x.add_module(bn_name, nn.BatchNorm2d(output_size))

        conv_o_h = nn.Conv2d(output_size, output_size, h_kernel_size, stride=1, padding=pad_h)
        batchnorm_o_h = nn.BatchNorm2d(output_size)
        self.o_h = nn.Sequential()
        self.o_h.add_module('conv_1_h', conv_o_h)
        self.o_h.add_module('bn_1_h', batchnorm_o_h)
        for i in range(num_cell-1):
            relu_name = 'relu' + str(i+1)
            conv_name = 'conv_{}_h'.format(str(i + 2))
            bn_name = 'bn_{}_h'.format(str(i + 2))
            self.o_h.add_module(relu_name, nn.ReLU())
            self.o_h.add_module(conv_name, nn.Conv2d(output_size, output_size, h_kernel_size, stride=1, padding=pad_h))
            self.o_h.add_module(bn_name, nn.BatchNorm2d(output_size))

        # init bias
        for i in range(num_cell):
            self.i_x[i * 3].bias.data.fill_(0)
            self.f_x[i * 3].bias.data.fill_(0)
            self.c_x[i * 3].bias.data.fill_(0)
            self.o_x[i * 3].bias.data.fill_(0)

        self.last_cell = None
        self.last_h = None
        
    def reset_state(self):
        self.last_cell = None
        self.last_h = None
    
    def forward(self, x):
        if self.last_cell is None:
            self.last_cell = Variable(torch.zeros(
                (x.size(0), self.output_size, int(x.size(2)/self.stride), 
                 int(x.size(3)/self.stride))
            ))
            if x.is_cuda:
                self.last_cell = self.last_cell.cuda()
        if self.last_h is None:
            self.last_h = Variable(torch.zeros(
                (x.size(0), self.output_size, int(x.size(2)/self.stride), 
                 int(x.size(3)/self.stride))
            ))
            if x.is_cuda:
                self.last_h = self.last_h.cuda()
        h = self.last_h
        c = self.last_cell
        
       
        input_x = self.i_x(x)
        input_h = self.i_h(h)
        input_gate = torch.sigmoid(input_x + input_h)
        
        
        forget_x = self.f_x(x)
        forget_h = self.f_h(h)
        forget_gate = torch.sigmoid(forget_x + forget_h)
        
       
        cell_x = self.c_x(x)
        cell_h = self.c_h(h)
        cell_intermediate = torch.tanh(cell_x + cell_h) # g
        cell_gate = (forget_gate * c) + (input_gate * cell_intermediate)
       
        output_x = self.o_x(x)
        output_h = self.o_h(h)
        output_gate = torch.sigmoid(output_x + output_h)
        
        next_h = output_gate * torch.tanh(cell_gate)
        self.last_cell = cell_gate
        self.last_h = next_h
        
        return next_h

# Definition of the FeedbackConvLSTM

In [ ]:
import torch 
import pytorch_lightning as pl

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import math

class FeedbackConvLSTM(pl.LightningModule):
    def __init__(self, input_size, output_sizes, strides, num_iterations, x_kernel_size, h_kernel_size):
        super(FeedbackConvLSTM, self).__init__()
        
        assert len(output_sizes) == len(strides)
        
        self.physical_depth = len(output_sizes)
        self.num_iterations = num_iterations
        self.input_size = input_size
        self.output_sizes = output_sizes
        self.strides = strides
        self.x_kernel_size = x_kernel_size
        self.h_kernel_size = h_kernel_size
        
        self.convlstm_cells = nn.ModuleList()
        for it in range(self.physical_depth):
            if it == 0:
                inp_size = input_size
            else:
                inp_size = output_sizes[it-1]
            outp_size = output_sizes[it]
            stride = strides[it]
            
            self.convlstm_cells.append(
                ConvLSTMCell(inp_size, outp_size, x_kernel_size, h_kernel_size, stride)
            )
        
    def forward(self, x):
        end_xts = []
        for t in range(self.num_iterations):
            for d in range(self.physical_depth):
                if d == 0:
                    x_t = x # x_t^{d-1}
                x_t = self.convlstm_cells[d].forward(x_t)
            end_xts.append(x_t)

        # prevent memory leak
        for cell in self.convlstm_cells:
            cell.reset_state()

        return end_xts

class FeedbackModule(pl.LightningModule):
    # physical layers = list of modules, must implement reset_state() method
    # num_iterations, repeat n times, for physical_depth*num_iterations -> virtual_depth
    def __init__(self, physical_layers, num_iterations):
        super(FeedbackModule, self).__init__()
        self.cells = nn.ModuleList(physical_layers)
        self.num_iterations = num_iterations

    def forward(self, x):
        end_xts = []
        for t in range(self.num_iterations):
            x_t = x
            for cell in self.cells:
                x_t = cell(x_t)#.forward(x_t)
            end_xts.append(x_t)

        for cell in self.cells:
            cell.reset_state()

        return end_xts

# Definition of the ConvLSTMStack

In [ ]:
import torch


from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import math

class ConvLSTMStack(pl.LightningModule):
    
    def __init__(self, input_size, output_size, x_kernel_size, h_kernel_size, stride, n):        
        super(ConvLSTMStack, self).__init__()
        operations = []
        self.cells = nn.ModuleList()
        first_cell = ConvLSTMCell(input_size, output_size, x_kernel_size, h_kernel_size, stride, n)
        self.cells.append(first_cell)
        operations.append(first_cell)
       
        self.stack = nn.Sequential(*operations)

        
    def reset_state(self):
        for cell in self.cells:
            cell.reset_state()
    
    def forward(self, x):
        return self.stack(x)

# Definition of the FeedbackNet

In [ ]:
init_lr = 0.6  # 0.6
lr_decay = 0.5
momentum = 0.9
weight_decay = 1.e-4
nesterov = True

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import math
from pytorch_lightning.metrics.functional import accuracy
import os
import torch as t
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.autograd import Variable




class FeedbackNet(pl.LightningModule):  # 12 physical depth, 4 iterations, n_classes output classes
    def __init__(self, n_classes):
        super(FeedbackNet, self).__init__()
        self.n_classes = n_classes
        self.num_iterations = 4
        self.init_lr=0.6
        self.lr_decay = 0.5
        self.momentum = 0.9
        self.weight_decay = 1.e-4
        self.nesterov = True
        self.criterion = nn.CrossEntropyLoss()
        self.gamma = 1
        self.conv = nn.Conv2d(3, 16, 3, 1, 1)
        self.batchnorm = nn.BatchNorm2d(16)

        #Note: hparams is logged to wandb as the config, so you'll never lose track of the arguments you used to run a model again!
        # log hyperparameters <=> capture tous les hyperparamètres et aujoute les au dictionnaire self.hparams
        self.save_hyperparameters()

        # compute the accuracy -- no need to roll your own!
        self.train_acc = pl.metrics.Accuracy()
        self.valid_acc = pl.metrics.Accuracy()

        stack = [
            ConvLSTMStack(16, 16, 3, 3, 1, 3),  # Iterate(fi,fo,kernel,kernel,stride,conv_num_per_stack)
            ConvLSTMStack(16, 32, 3, 3, 2, 3),
            ConvLSTMStack(32, 64, 3, 3, 2, 3),
            ConvLSTMStack(64, 64, 3, 3, 1, 3),
        ]
        self.feedback = FeedbackModule(stack, self.num_iterations)
        self.avg_pool = nn.AvgPool2d(8)#4)
        self.dropout1 = nn.Dropout(p=0.5)
        self.linear = nn.Linear(256, 256)
        self.dropout2 = nn.Dropout(p=0.5)
        # self.output = nn.Linear(256, self.n_classes)#n_output_classes)
        self.output = nn.Linear(64, self.n_classes)
        print('Initializing FeedbackNet48!')

        
    def training_step(self,batch,batch_idx):
      inputs,target=batch 
      loss = Variable(t.from_numpy(np.zeros(1))).float()
      outputs = self(inputs)
      losses = [self.criterion(out, target) for out in outputs]
      for it in range(len(losses)):
       loss += (self.gamma ** it) * losses[it]

      self.log("valid/loss_epoch", loss,on_step=True, on_epoch=True)
      self.log('train/acc', self.train_acc, on_step=True,on_epoch=True)
      
      return {'loss':loss}

    def configure_optimizers(self):
      return torch.optim.SGD(self.parameters(), lr=self.init_lr,
                                   momentum=self.momentum,
                                   weight_decay=self.weight_decay,
                                   nesterov=self.nesterov)

    def validation_step(self,batch,batch_idx):
      inputs,target=batch 
      loss = Variable(t.from_numpy(np.zeros(1))).float()
      # forward
      outputs = self(inputs)
      #self.valid_acc(outputs, target)
      losses = [self.criterion(out, target) for out in outputs]
      for it in range(len(losses)):
        loss += (self.gamma ** it) * losses[it]
      self.log("valid/loss_epoch", loss,on_step=True, on_epoch=True)  
      self.log('valid/acc_epoch', self.valid_acc,on_step=True, on_epoch=True)
      

    
      return outputs
      
    

    def validation_epoch_end(self, validation_step_outputs):
      #1 save the model (1,3,32,32)
      dummy_input = torch.zeros((1,3,32,32), device=self.device)
      model_filename = "model_final.onnx"
      torch.onnx.export(self, dummy_input, model_filename)
      wandb.save(model_filename) #comment this line before trying compling

      #2 turn into histogramm
      #flattened_logits = torch.flatten(torch.cat(validation_step_outputs))
      #self.logger.experiment.log(
       # {"valid/logits": wandb.Histogram(flattened_logits.to("cpu")),
       #  "global_step": self.global_step})
      

    
    def forward(self, x):
      x = self.conv(x)  
      x = self.batchnorm(x)
      x = F.relu(x)
      x_all = self.feedback(x)
      x_finished = []
      for x_i in x_all:
          x_i = F.relu(x_i)
          x_i = self.avg_pool(x_i)
          
          x_i = x_i.view(-1, 64)
          x_i = self.output(x_i)
          x_finished.append(x_i)
     
      return x_finished
      
      
      

# Definition Of the FeedbackNetDataModule

In [ ]:
def get_transform(is_train=True):

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    if is_train:
        return transforms.Compose(
            [transforms.Pad(4),
             transforms.RandomResizedCrop(32),
             transforms.RandomHorizontalFlip(),
             transforms.ToTensor(),
             normalize]
        )
    else:
        return transforms.Compose(
            [transforms.ToTensor(),
             normalize]
        )

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

class FeedbackNetDataModule(pl.LightningDataModule):

  def setup(self, stage=None):
     
    self.cifar10_train_data = datasets.CIFAR10("./data", train=True, download=True, transform=get_transform(is_train=True))
    self.cifar10_test_data = datasets.CIFAR10("./data", train=False, download=True, transform=get_transform(is_train=False))
    #self.cifar100_train_data = datasets.CIFAR100("./data", train=True, download=True, transform=get_transform(is_train=True))
    #self.cifar100_test_data = datasets.CIFAR100("./data", train=False, download=True, transform=get_transform(is_train=False))

   

  def train_dataloader(self):
    return DataLoader(self.cifar10_train_data, batch_size=64)

  def val_dataloader(self):
    return DataLoader(self.cifar10_test_data, batch_size=64)

  

## TRY COMPILING FEEDBACKNET


In [ ]:
#try to compile
# models
model = FeedbackNet(10)
data_module = FeedbackNetDataModule()

trainer=pl.Trainer(fast_dev_run=True)
trainer.fit(model,data_module)


GPU available: False, used: False
TPU available: None, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).


Initializing FeedbackNet48!
Files already downloaded and verified
Files already downloaded and verified



   | Name      | Type             | Params
------------------------------------------------
0  | criterion | CrossEntropyLoss | 0     
1  | conv      | Conv2d           | 448   
2  | batchnorm | BatchNorm2d      | 32    
3  | train_acc | Accuracy         | 0     
4  | valid_acc | Accuracy         | 0     
5  | feedback  | FeedbackModule   | 2.0 M 
6  | avg_pool  | AvgPool2d        | 0     
7  | dropout1  | Dropout          | 0     
8  | linear    | Linear           | 65.8 K
9  | dropout2  | Dropout          | 0     
10 | output    | Linear           | 650   
------------------------------------------------
2.0 M     Trainable params
0         Non-trainable params
2.0 M     Total params
8.134     Total estimated model params size (MB)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:148: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:149: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:155: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
/usr/lo

1

## Making a trainer

In [ ]:
%%capture
!pip install wandb --upgrade

In [ ]:
from pytorch_lightning.loggers import WandbLogger

In [ ]:
wandb_logger = WandbLogger(project="by_Chimi_Adda_leo_feedbackNet_efrei_BigData_ML")

In [ ]:
trainer = pl.Trainer(
    logger=wandb_logger,    # W&B integration
    log_every_n_steps=50,   # set the logging frequency
    gpus=0,                  
    max_epochs=2,           # number of epochs
    deterministic=True,     # keep it deterministic
    num_sanity_val_steps=0
    )

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


# Running our FeedbackNET Model

In [ ]:
# models
model = FeedbackNet(10)
data_module = FeedbackNetDataModule()
trainer.fit(model,data_module)
wandb.finish()

Initializing FeedbackNet48!
Files already downloaded and verified
Files already downloaded and verified



   | Name      | Type             | Params
------------------------------------------------
0  | criterion | CrossEntropyLoss | 0     
1  | conv      | Conv2d           | 448   
2  | batchnorm | BatchNorm2d      | 32    
3  | train_acc | Accuracy         | 0     
4  | valid_acc | Accuracy         | 0     
5  | feedback  | FeedbackModule   | 2.0 M 
6  | avg_pool  | AvgPool2d        | 0     
7  | dropout1  | Dropout          | 0     
8  | linear    | Linear           | 65.8 K
9  | dropout2  | Dropout          | 0     
10 | output    | Linear           | 650   
------------------------------------------------
2.0 M     Trainable params
0         Non-trainable params
2.0 M     Total params
8.134     Total estimated model params size (MB)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Trying to log at a previous step. Use `WandbLogger(sync_step=False)` or try logging with `commit=False` when calling manually `wandb.log`.
  warnings.warn(*args, **kwargs)
wandb: WARNING Step must only increase in log calls.  Step 0 < 781; dropping {'valid/loss_epoch_step/epoch_0': 7.796355247497559, 'valid/acc_epoch_step/epoch_0': Accuracy()}.
wandb: WARNING Step must only increase in log calls.  Step 1 < 781; dropping {'valid/loss_epoch_step/epoch_0': 7.519670486450195, 'valid/acc_epoch_step/epoch_0': Accuracy()}.
wandb: WARNING Step must only increase in log calls.  Step 2 < 781; dropping {'valid/loss_epoch_step/epoch_0': 7.347436428070068, 'valid/acc_epoch_step/epoch_0': Accuracy()}.
wandb: WARNING Step must only increase in log calls.  Step 3 < 781; dropping {'valid/loss_epoch_step/epoch_0': 7.877077102661133, 'valid/acc_epoch_step/epoch_0': Accuracy()}.
wandb: WARNING Step must only

wandb: WARNING Step must only increase in log calls.  Step 0 < 1563; dropping {'valid/loss_epoch_step/epoch_1': 7.666613578796387, 'valid/acc_epoch_step/epoch_1': Accuracy()}.
wandb: WARNING Step must only increase in log calls.  Step 1 < 1563; dropping {'valid/loss_epoch_step/epoch_1': 6.3993330001831055, 'valid/acc_epoch_step/epoch_1': Accuracy()}.
wandb: WARNING Step must only increase in log calls.  Step 2 < 1563; dropping {'valid/loss_epoch_step/epoch_1': 7.0729169845581055, 'valid/acc_epoch_step/epoch_1': Accuracy()}.
wandb: WARNING Step must only increase in log calls.  Step 3 < 1563; dropping {'valid/loss_epoch_step/epoch_1': 7.669988632202148, 'valid/acc_epoch_step/epoch_1': Accuracy()}.
wandb: WARNING Step must only increase in log calls.  Step 4 < 1563; dropping {'valid/loss_epoch_step/epoch_1': 6.731636047363281, 'valid/acc_epoch_step/epoch_1': Accuracy()}.
wandb: WARNING Step must only increase in log calls.  Step 5 < 1563; dropping {'valid/loss_epoch_step/epoch_1': 7.4381

valid/loss_epoch_step,6.59229
train/acc_step,Accuracy()
epoch,1
_runtime,19446
_timestamp,1614258316
_step,1563
valid/loss_epoch_epoch,7.06364
train/acc_epoch,nan
valid/acc_epoch_epoch,nan


valid/loss_epoch_step,█▅▅▆▅▆▄▆▃▃▃▆▃▄▂▂▂▃▃▁▃▂▂▄▃▂▂▁▂▂▁
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█████████████████
_runtime,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇████
_timestamp,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇████
_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇███
valid/loss_epoch_epoch,█▁
train/acc_epoch,
valid/acc_epoch_epoch,
